### wtfPython docu

## 1.string interning

In [1]:
a="a_b"
b="a"+"_"+"b"
print(id(a)==id(b))

True


In [2]:
a=a+"!"
b=b+"!"
print(id(a) is id(b))

False


In [3]:
a=''.join(['f','o','o'])
b=''.join(['f','o','o'])
a is  b

False

Such behavior is due to Cython optimization called **`string interning`** that tries to use the existing immutable objects rather than creating a new object to save memory.
- all length 0 and 1 string are interned
- strings will be interned at compile time instead of run time 
- strings that are not composed of ASCII letters ,digits or underscores are not interned
> by the way **the different of 'id' and '=='**<br>
'id' checks if both the operands refer to the same object .it is for reference equality<br>
'==' compares the values of both the operands and checks if they are the same. it is for the value equality
------------------
这是因为Cpython优化的**字符串驻留**机制,其尝试使用已存在的不可变变量而非新建一个变量
- 只有长度为0或1的字符会被驻留
- 字符串驻留会在编译期发生即替换结果而非运行时
- 字符串中字母、数字、下划线不会被驻留
> 顺便提及‘id’ 和‘==’的区别  
‘id’判断地址 ‘==’判断值

## 2.peephole optimization  constant floding  small_ints cache

In [4]:
import dis
def foo():
    return 'foo'+'bar'
dis.dis(foo)

  3           0 LOAD_CONST               3 ('foobar')
              2 RETURN_VALUE


In [5]:
a='a'*20
b='a'*21
a is b

False

In [6]:
a,b=257,257
print(a==b)
print(a is b)
print(id(a)==id(b))
a=257
b=257
print(a==b)
print(a is b)
print(id(a)==id(b))

True
True
True
True
False
False


- source code will be transformed to **`raw byte code`** first than transformed to 'more efficient byte code ' via peephole optimization  
- **`constant folding`** only occurs strings having length less than 20 (wont let **`.pyc`** to large) to reduce few clock cycles during runtime  
- python will cache the frequency number to a list named **`small_ints`** just for **`[-5,256]`**  
' = ' assigning operator in the same line will allocate same memory if just one value for multi-variances 
-----------
- python源码会先被编译成原始字节码进而再通过peephole优化编译成更加高效的字节码
- **字符串折叠** 仅仅发生在字符串长度不超过20的前提下(否则会使pyc文件过大),以在运行期减少时间周期
- python会缓存常用整数 范围仅(-5,256)
- 同行赋值同值同地址

## 3.dict K&V

In [8]:
dict1[1]='1'
dict1[1.0]='2'
dict1[1.1]='3'
print(dict1[1])
print(dict1[1.0])
print(dict1[1.1])

NameError: name 'dict1' is not defined

In [ ]:
hash(1.0) is hash(1)

python dict check for equality and compare `hash(key)` to determinate if two keys have same value  

----------------
python 字典会根据`key`的hash值来确定是否是同一个key且在python中不可变变量同值同地址

## 4.immutable ＆ mutable reference

In [ ]:
b=1
print(id(b))
b=2
print(id(b))
c=1
print(id(c))
a=[1,2]
c=[a]*3
print(c,' ',id(c[0]),' ',id(a[0]))
a[0]=3
print(c,' ',id(c[0]),' ',id(a[0]))

immutable varience will give certain memory by value and this memory cant be changed just can turn to other memory<br>
mutable varience will give certain memory and the deeply immutable varience in this are same as below  

---------------------------------------
h不可变变量根据值分配内存地址若更改即更改其指向
可变变量分配内存地址后不变,子不可变变量法则如上

## 5. declare & runtime

In [ ]:
array=[1,8,15]
g=(i for i in array if array.count(i)>0)
array=[2,8,22]
print(list(g))
array=[1,8,15]
g=[i for i in array if array.count(i)>0]
array=[2,8,22]
print(g)

In　a **`generator`** expression __`in`__ is declareation time to determinate some constant or others, but the __`if`__ conditional clause is evaluated at runtime  
SO first **x** is determined in compile time by '1,8,15' second **x** will determined in runtime  
And second line express a generator it wont excute right away  
When call **list()** to excute generator **array** is changed to '2,8,22' 

--------------------------------------------------

在生成器表达式中`in`关键字是在编译期决定`x`的值，而`if`则是在运行期决定array的值
所以在for中的x为[1,8,15]

## 6.

In [ ]:
funcs = []
results = []
for x in range(7):
    def some_func():
        return x
    funcs.append(some_func)
    results.append(some_func())
funcs_results = [func() for func in funcs]
print(results,'\n',funcs_results)

In [ ]:
func=[lambda x: x**i for i in range(7)]
[f(2) for f in func]

when defining a function inside a loop which uses variables in its body, the loop function is bound to the variable's memory not the value. So all of the functions use the latest value assigned to the variable for computation

-----------------------------
循环内部定义的函数使用的变量的话，函数绑定其指针而非地址，且该例中多个不同的函数绑定的是同一个x变量的地址  
所以调用时的x都是最新的x值